In [1]:
import json
import joblib
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix
from plotly.subplots import make_subplots

In [2]:
def load_and_predict(test_file, model_file, cols_file):
    """Carrega dados, modelo e gera y_true e y_pred"""
    
    # 1. Carrega colunas e Modelo
    cols_info = joblib.load(cols_file)
    input_cols = cols_info['input']
    target_cols = cols_info['target']
    pipeline = joblib.load(model_file)
    
    # 2. Carrega o JSON de Teste
    with open(test_file, 'r') as f:
        data = json.load(f)
    
    # 3. Processamento (Flattening)
    flat_data = []
    for sample in data:
        row = {}
        inputs = sample['inputs']
        outputs = sample['outputs']
        
        # --- INPUTS ---
        for k, v in inputs['thermal_gen_pu'].items(): row[f'in_gen_{k}'] = v
        for k, v in inputs['wind_available_pu'].items(): row[f'in_wind_{k}'] = v
        for k, v in inputs['load_required'].items():
            row[f'in_load_P_{k}'] = v['p_pu']
            row[f'in_load_Q_{k}'] = v['q_pu']

        # --- OUTPUTS (TARGETS) ---
        
        # AQUI ESTAVA FALTANDO ESTE BLOCO:
        if 'wind_curtailment_pu' in outputs:
            for k, v in outputs['wind_curtailment_pu'].items(): row[f'out_curt_{k}'] = v
            
        if 'load_deficit_pu' in outputs:
            for k, v in outputs['load_deficit_pu'].items(): row[f'out_def_{k}'] = v
            
        if 'thermal_gen_q_pu' in outputs:
            for k, v in outputs['thermal_gen_q_pu'].items(): row[f'out_gen_Q_{k}'] = v
        
        # Carregamento Linhas
        for key, val in outputs.items():
            if key.startswith('line_loading_'):
                clean_name = key.replace('line_loading_', '').replace('_pu', '')
                row[f'out_loading_{clean_name}'] = val
        
        flat_data.append(row)

    df_test = pd.DataFrame(flat_data)
    
    # 4. Alinhamento de Colunas (Preenche faltantes com 0)
    for col in input_cols + target_cols:
        if col not in df_test.columns: df_test[col] = 0.0

    # 5. Predição
    print("Realizando predição na Rede Neural...")
    y_true = df_test[target_cols]
    y_pred_vals = pipeline.predict(df_test[input_cols])
    y_pred = pd.DataFrame(y_pred_vals, columns=target_cols, index=df_test.index)
    
    # DEBUG: Para confirmar que agora leu certo
    curt_cols = [c for c in target_cols if 'out_curt_' in c]
    if curt_cols:
        print(f"Máximo Corte Eólico Real encontrado: {y_true[curt_cols].max().max():.4f} pu")
    
    return y_true, y_pred, target_cols

In [3]:
def plot_shedding_confusion(y_true, y_pred, target_cols):
    def_cols = [c for c in target_cols if 'out_def_' in c]
    if not def_cols: return print("Sem colunas de déficit.")

    # Lógica Binária
    threshold = 1e-3
    real_event = (y_true[def_cols].sum(axis=1) > threshold).astype(int)
    pred_event = (y_pred[def_cols].sum(axis=1) > threshold).astype(int)
    
    # Matriz [1, 0] -> Índice 0 é o Evento Crítico
    cm = confusion_matrix(real_event, pred_event, labels=[1, 0])
    
    # CÁLCULO DAS PORCENTAGENS
    # Normaliza pelo total de amostras para ter % global
    total = np.sum(cm)
    cm_perc = (cm / total * 100)
    
    # Textos formatados: "XX.X% <br> (N amostras)"
    z_text = [
        [f"<b>{cm_perc[0][0]:.1f}%</b><br>({cm[0][0]})<br>Acerto: Corte", 
         f"<b>{cm_perc[0][1]:.1f}%</b><br>({cm[0][1]})<br>ERRO: Perdeu Corte"],
         
        [f"<b>{cm_perc[1][0]:.1f}%</b><br>({cm[1][0]})<br>Alarme Falso", 
         f"<b>{cm_perc[1][1]:.1f}%</b><br>({cm[1][1]})<br>Acerto: Normal"]
    ]
    
    fig = ff.create_annotated_heatmap(
        z=cm, 
        x=['Pred: Crítico', 'Pred: Seguro'], 
        y=['Real: Crítico', 'Real: Seguro'], 
        annotation_text=z_text, 
        colorscale='Reds', 
        showscale=False
    )
    
    fig.update_layout(title_text="<b>Matriz de Confusão:</b> Detecção de Load Shedding (%)", 
                      height=500, width=600)
    fig.show()

def plot_line_loading_scatter(y_true, y_pred, target_cols):
    load_cols = [c for c in target_cols if 'out_loading_' in c]
    if not load_cols: return print("Sem colunas de carregamento.")

    # Achata tudo para 1D
    y_true_flat = y_true[load_cols].values.flatten()
    y_pred_flat = y_pred[load_cols].values.flatten()
    
    # Filtra apenas carregamentos relevantes (> 40%)
    mask = y_true_flat > 0.4
    real_f = y_true_flat[mask]
    pred_f = y_pred_flat[mask]
    
    fig = go.Figure()
    # Linha Ideal
    fig.add_trace(go.Scatter(x=[0, 1.2], y=[0, 1.2], mode='lines', line=dict(color='black', dash='dash'), name='Ideal'))
    # Pontos
    fig.add_trace(go.Scatter(x=real_f, y=pred_f, mode='markers', 
                             marker=dict(color=real_f, colorscale='Viridis', showscale=True, size=6),
                             name='Amostras'))

    # Zona de Saturação
    fig.add_shape(type="rect", x0=0.95, y0=0.95, x1=1.05, y1=1.05, line=dict(color="red", dash="dot"))
    fig.add_annotation(x=1.0, y=1.1, text="Zona de Saturação", showarrow=False, font=dict(color="red"))

    fig.update_layout(title="<b>Regressão:</b> Carregamento de Linhas (>0.4 pu)",
                      xaxis_title="Real (Solver)", yaxis_title="Predito (RNA)",
                      height=600, width=800)
    fig.show()

In [4]:
def plot_curtailment_confusion(y_true, y_pred, target_cols):
    curt_cols = [c for c in target_cols if 'out_curt_' in c]
    if not curt_cols: return print("Sem colunas de corte eólico.")

    threshold = 1e-3
    real_event = (y_true[curt_cols].sum(axis=1) > threshold).astype(int)
    pred_event = (y_pred[curt_cols].sum(axis=1) > threshold).astype(int)
    
    # Labels=[1, 0] garante que o índice 0 seja o evento positivo (Com Corte)
    cm = confusion_matrix(real_event, pred_event, labels=[1, 0])
    
    # CÁLCULO DAS PORCENTAGENS
    total = np.sum(cm)
    cm_perc = (cm / total * 100)
    
    # Textos Atualizados para clareza
    z_text = [
        [f"<b>{cm_perc[0][0]:.1f}%</b><br>({cm[0][0]})<br>Acerto: Teve Corte", 
         f"<b>{cm_perc[0][1]:.1f}%</b><br>({cm[0][1]})<br>ERRO: Não viu"],
         
        [f"<b>{cm_perc[1][0]:.1f}%</b><br>({cm[1][0]})<br>Alarme Falso", 
         f"<b>{cm_perc[1][1]:.1f}%</b><br>({cm[1][1]})<br>Acerto: Sem Cortes"]
    ]
    
    fig = ff.create_annotated_heatmap(
        z=cm, 
        x=['Pred: Com Corte', 'Pred: Sem Cortes'], 
        y=['Real: Com Corte', 'Real: Sem Cortes'], 
        annotation_text=z_text, 
        colorscale='Oranges', 
        showscale=False
    )
    
    fig.update_layout(title_text="<b>Matriz de Confusão:</b> Detecção de Curtailment (%)", 
                      height=500, width=600)
    fig.show()

def plot_curtailment_scatter(y_true, y_pred, target_cols):
    # Identifica colunas
    curt_cols = [c for c in target_cols if 'out_curt_' in c]
    if not curt_cols: return

    # Achata para 1D (todos os geradores eólicos juntos)
    y_true_flat = y_true[curt_cols].values.flatten()
    y_pred_flat = y_pred[curt_cols].values.flatten()
    
    # Filtra: Só queremos ver quando houve algum corte relevante (> 0.01 pu)
    mask = y_true_flat > 0.01
    
    # Se não tiver dados suficientes filtrados, plota tudo
    if sum(mask) < 10: 
        real_f = y_true_flat
        pred_f = y_pred_flat
    else:
        real_f = y_true_flat[mask]
        pred_f = y_pred_flat[mask]
    
    fig = go.Figure()

    # Linha Ideal
    fig.add_trace(go.Scatter(
        x=[0, max(real_f.max(), 1.0)], y=[0, max(real_f.max(), 1.0)], 
        mode='lines', line=dict(color='black', dash='dash'), name='Ideal'
    ))

    # Pontos
    fig.add_trace(go.Scatter(
        x=real_f, y=pred_f,
        mode='markers',
        marker=dict(
            size=7, 
            color=real_f, 
            colorscale='Oranges', # Combina com a matriz
            showscale=True,
            line=dict(width=1, color='DarkSlateGrey')
        ),
        name='Amostras',
        hovertemplate="Real: %{x:.3f} pu<br>Pred: %{y:.3f} pu"
    ))

    fig.update_layout(
        title_text="<b>Regressão:</b> Precisão da Estimativa de Corte Eólico",
        title_x=0.5,
        xaxis_title="Corte Real (Solver AC) [pu]",
        yaxis_title="Corte Predito (RNA) [pu]",
        height=600, width=700,
        template="plotly_white"
    )

    fig.show()
    # Identifica colunas
    curt_cols = [c for c in target_cols if 'out_curt_' in c]
    if not curt_cols: return

    # Achata para 1D (todos os geradores eólicos juntos)
    y_true_flat = y_true[curt_cols].values.flatten()
    y_pred_flat = y_pred[curt_cols].values.flatten()
    
    # Filtra: Só queremos ver quando houve algum corte relevante (> 0.01 pu)
    mask = y_true_flat > 0.01
    
    # Se não tiver dados suficientes filtrados, plota tudo
    if sum(mask) < 10: 
        real_f = y_true_flat
        pred_f = y_pred_flat
    else:
        real_f = y_true_flat[mask]
        pred_f = y_pred_flat[mask]
    
    fig = go.Figure()

    # Linha Ideal
    fig.add_trace(go.Scatter(
        x=[0, max(real_f.max(), 1.0)], y=[0, max(real_f.max(), 1.0)], 
        mode='lines', line=dict(color='black', dash='dash'), name='Ideal'
    ))

    # Pontos
    fig.add_trace(go.Scatter(
        x=real_f, y=pred_f,
        mode='markers',
        marker=dict(
            size=7, 
            color=real_f, 
            colorscale='Oranges', # Combina com a matriz
            showscale=True,
            line=dict(width=1, color='DarkSlateGrey')
        ),
        name='Amostras',
        hovertemplate="Real: %{x:.3f} pu<br>Pred: %{y:.3f} pu"
    ))

    fig.update_layout(
        title_text="<b>Regressão:</b> Precisão da Estimativa de Corte Eólico",
        title_x=0.5,
        xaxis_title="Corte Real (Solver AC) [pu]",
        yaxis_title="Corte Predito (RNA) [pu]",
        height=600, width=700,
        template="plotly_white"
    )

    fig.show()

In [5]:
# Caminhos dos arquivos (devem estar na mesma pasta do .ipynb)
TEST_FILE = "data_test.json"
MODEL_FILE = "rna_proxy_model.pkl"
COLS_FILE = "rna_cols.pkl"

try:
    # 1. Carrega e Prediz
    y_t, y_p, cols = load_and_predict(TEST_FILE, MODEL_FILE, COLS_FILE)
        
    print("\n--- ANÁLISE DE OPERAÇÃO EÓLICA (CURTAILMENT) ---")
    plot_curtailment_confusion(y_t, y_p, cols) # <--- NOVA
    plot_curtailment_scatter(y_t, y_p, cols)   # <--- NOVA (Opcional, mas recomendada)
    
    print("\n--- ANÁLISE DE FLUXO ---")
    plot_line_loading_scatter(y_t, y_p, cols)

    print("\n--- ANÁLISE DE SEGURANÇA (LOAD SHEDDING) ---")
    plot_shedding_confusion(y_t, y_p, cols)

except FileNotFoundError as e:
    print(f"ERRO: {e}")

Realizando predição na Rede Neural...
Máximo Corte Eólico Real encontrado: 0.4881 pu

--- ANÁLISE DE OPERAÇÃO EÓLICA (CURTAILMENT) ---



--- ANÁLISE DE FLUXO ---



--- ANÁLISE DE SEGURANÇA (LOAD SHEDDING) ---


In [6]:
def load_and_predict(test_file, model_file, cols_file):
    """Carrega dados, modelo e gera y_true e y_pred"""
    
    # 1. Carrega colunas e Modelo
    cols_info = joblib.load(cols_file)
    input_cols = cols_info['input']
    target_cols = cols_info['target']
    pipeline = joblib.load(model_file)
    
    # 2. Carrega o JSON de Teste
    with open(test_file, 'r') as f:
        data = json.load(f)
    
    # 3. Processamento (Flattening)
    flat_data = []
    for sample in data:
        row = {}
        inputs = sample['inputs']
        outputs = sample['outputs']
        
        # --- INPUTS ---
        for k, v in inputs['thermal_gen_pu'].items(): row[f'in_gen_{k}'] = v
        for k, v in inputs['wind_available_pu'].items(): row[f'in_wind_{k}'] = v
        for k, v in inputs['load_required'].items():
            row[f'in_load_P_{k}'] = v['p_pu']
            row[f'in_load_Q_{k}'] = v['q_pu']

        # --- OUTPUTS (TARGETS) ---
        
        # 1. Tensão (ESTAVA FALTANDO ISSO AQUI PROVAVELMENTE)
        if 'bus_voltage_pu' in outputs:
            for k, v in outputs['bus_voltage_pu'].items(): row[f'out_v_{k}'] = v

        # 2. Cortes Eólicos
        if 'wind_curtailment_pu' in outputs:
            for k, v in outputs['wind_curtailment_pu'].items(): row[f'out_curt_{k}'] = v
        
        # 3. Déficit de Carga
        if 'load_deficit_pu' in outputs:
            for k, v in outputs['load_deficit_pu'].items(): row[f'out_def_{k}'] = v
        
        # 4. Geração Reativa
        if 'thermal_gen_q_pu' in outputs:
            for k, v in outputs['thermal_gen_q_pu'].items(): row[f'out_gen_Q_{k}'] = v
        
        # 5. Carregamento Linhas
        for key, val in outputs.items():
            if key.startswith('line_loading_'):
                clean_name = key.replace('line_loading_', '').replace('_pu', '')
                row[f'out_loading_{clean_name}'] = val
        
        flat_data.append(row)

    df_test = pd.DataFrame(flat_data)
    
    # 4. Alinhamento de Colunas (Preenche faltantes com 0)
    for col in input_cols + target_cols:
        if col not in df_test.columns: df_test[col] = 0.0

    # 5. Predição
    print("Realizando predição na Rede Neural...")
    y_true = df_test[target_cols]
    y_pred_vals = pipeline.predict(df_test[input_cols])
    y_pred = pd.DataFrame(y_pred_vals, columns=target_cols, index=df_test.index)
    
    # DEBUG: Verifica se leu as tensões corretamente agora
    volt_cols = [c for c in target_cols if 'out_v_' in c]
    if volt_cols:
        media_tensao = y_true[volt_cols].mean().mean()
        print(f"DEBUG: Média das Tensões Reais lidas: {media_tensao:.4f} pu (Deve ser próximo de 1.0)")
        if media_tensao < 0.1:
            print("ALERTA: As tensões ainda estão vindo como Zero! Verifique o nome das chaves no JSON.")
    
    return y_true, y_pred, target_cols

In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error

def plot_error_summary_table(y_true, y_pred, target_cols):
    """
    Calcula o MAE (Erro Médio Absoluto) para cada grupo de variáveis
    e plota uma tabela formatada para apresentação.
    """
    
    # 1. Definição dos Grupos de Variáveis
    groups = {
        'Curtailment (Corte Eólico)': [c for c in target_cols if 'out_curt_' in c],
        'Geração Reativa (Q)':        [c for c in target_cols if 'out_gen_Q_' in c],
        'Carregamento das LTs':       [c for c in target_cols if 'out_loading_' in c],
        'Tensões (Magnitude)':        [c for c in target_cols if 'out_v_' in c]
    }
    
    results = []
    
    # 2. Cálculo dos Erros
    for name, cols in groups.items():
        if not cols:
            results.append([name, "N/A"])
            continue
            
        y_t_sub = y_true[cols].values
        y_p_sub = y_pred[cols].values
        
        # Calcula MAE Global
        mae = mean_absolute_error(y_t_sub, y_p_sub)
        
        # Formata
        results.append([name, f"{mae:.5f} pu"])

    # Cria o DataFrame com o nome da coluna completo
    df_res = pd.DataFrame(results, columns=["Grandeza", "Erro Médio (MAE)"])

    # 3. Plot da Tabela
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['<b>Grandeza</b>', '<b>Erro Médio</b>'], # Título visual (pode ser curto)
            line_color='white',
            fill_color='tomato', 
            align='center',
            font=dict(color='white', size=16, family="Arial")
        ),
        cells=dict(
            # AQUI ESTAVA O ERRO: O nome da coluna tem que bater com o DataFrame
            values=[df_res['Grandeza'], df_res['Erro Médio (MAE)']], 
            line_color='lightgrey',
            fill_color='white',
            align='center',
            font=dict(color='darkslategray', size=14, family="Arial"),
            height=40
        ))
    ])

    fig.update_layout(
        title_text="<b>Resultado B6L8: Precisão dos Estados (Teste)</b>",
        title_x=0.5,
        width=600, height=400,
        margin=dict(l=20, r=20, t=60, b=20)
    )
    
    fig.show()

# --- BLOCO DE EXECUÇÃO ---
try:
    plot_error_summary_table(y_t, y_p, cols)
except NameError:
    print("Por favor, rode a célula de 'load_and_predict' antes.")

In [9]:
import pandas as pd
import numpy as np
import json
import joblib

# Função de Carregamento (Cópia da versão corrigida para garantir)
def load_and_debug(test_file, model_file, cols_file):
    print(f"--- DIAGNÓSTICO DE TENSÃO ---")
    
    # 1. Carrega Colunas e Modelo
    cols_info = joblib.load(cols_file)
    input_cols = cols_info['input']
    target_cols = cols_info['target']
    pipeline = joblib.load(model_file)
    
    # 2. Leitura do JSON (Raw)
    with open(test_file, 'r') as f:
        data = json.load(f)
    
    # Verifica no JSON Bruto se existe 'bus_voltage_pu' na primeira amostra
    sample_0 = data[0]['outputs']
    print(f"Check JSON Bruto (Amostra 0): Tem 'bus_voltage_pu'? {'bus_voltage_pu' in sample_0}")
    if 'bus_voltage_pu' in sample_0:
        print(f"Exemplo valor bruto: {list(sample_0['bus_voltage_pu'].items())[0]}")
    else:
        print("ALERTA: 'bus_voltage_pu' NÃO ESTÁ no JSON de outputs!")

    # 3. Processamento para DataFrame
    flat_data = []
    for sample in data:
        row = {}
        # Inputs
        for k, v in sample['inputs']['thermal_gen_pu'].items(): row[f'in_gen_{k}'] = v
        for k, v in sample['inputs']['wind_available_pu'].items(): row[f'in_wind_{k}'] = v
        for k, v in sample['inputs']['load_required'].items():
            row[f'in_load_P_{k}'] = v['p_pu']; row[f'in_load_Q_{k}'] = v['q_pu']

        # Outputs
        outputs = sample['outputs']
        
        # --- O PONTO CRÍTICO: TENSÃO ---
        if 'bus_voltage_pu' in outputs:
            for k, v in outputs['bus_voltage_pu'].items(): row[f'out_v_{k}'] = v
            
        # Outros (Só pra não quebrar)
        if 'wind_curtailment_pu' in outputs:
            for k, v in outputs['wind_curtailment_pu'].items(): row[f'out_curt_{k}'] = v
        if 'load_deficit_pu' in outputs:
            for k, v in outputs['load_deficit_pu'].items(): row[f'out_def_{k}'] = v
        if 'thermal_gen_q_pu' in outputs:
            for k, v in outputs['thermal_gen_q_pu'].items(): row[f'out_gen_Q_{k}'] = v
        for k, v in outputs.items():
            if k.startswith('line_loading_'):
                clean = k.replace('line_loading_', '').replace('_pu', '')
                row[f'out_loading_{clean}'] = v
        
        flat_data.append(row)

    df = pd.DataFrame(flat_data)
    
    # Garante colunas
    for col in input_cols + target_cols:
        if col not in df.columns: df[col] = 0.0
            
    # 4. Predição
    y_true = df[target_cols]
    y_pred_vals = pipeline.predict(df[input_cols])
    y_pred = pd.DataFrame(y_pred_vals, columns=target_cols, index=df.index)
    
    # --- ANÁLISE ESTATÍSTICA ---
    volt_cols = [c for c in target_cols if 'out_v_' in c]
    
    if not volt_cols:
        print("ERRO: Nenhuma coluna de tensão (out_v_) encontrada nos targets do modelo!")
        return

    mean_real = y_true[volt_cols].mean().mean()
    mean_pred = y_pred[volt_cols].mean().mean()
    
    print(f"\nMédia Global das Tensões (Todas as Barras/Horas):")
    print(f"  REAL (JSON):    {mean_real:.4f} pu")
    print(f"  PREDITO (RNA):  {mean_pred:.4f} pu")
    
    if mean_real < 0.1:
        print("  -> CONFIRMADO: O vetor REAL está vindo zerado. O problema é na leitura ou no JSON.")
    else:
        print("  -> O vetor REAL parece ok. O erro pode ser outliers.")

    # --- AMOSTRA ALEATÓRIA ---
    idx = np.random.choice(df.index)
    print(f"\n--- Detalhes da Amostra Aleatória #{idx} ---")
    print(f"{'Barra':<15} | {'Real':<10} | {'Predito':<10} | {'Diferença':<10}")
    print("-" * 55)
    
    for col in volt_cols:
        v_real = y_true.loc[idx, col]
        v_pred = y_pred.loc[idx, col]
        diff = abs(v_real - v_pred)
        # Mostra apenas as primeiras 5 barras pra não poluir, ou todas se forem poucas
        print(f"{col:<15} | {v_real:.4f}     | {v_pred:.4f}     | {diff:.4f}")

# Executa
try:
    load_and_debug("data_test.json", "rna_proxy_model.pkl", "rna_cols.pkl")
except Exception as e:
    print(f"Erro no debug: {e}")

--- DIAGNÓSTICO DE TENSÃO ---
Check JSON Bruto (Amostra 0): Tem 'bus_voltage_pu'? True
Exemplo valor bruto: ('Bus_1', 1.042937459975038)

Média Global das Tensões (Todas as Barras/Horas):
  REAL (JSON):    1.0361 pu
  PREDITO (RNA):  1.0336 pu
  -> O vetor REAL parece ok. O erro pode ser outliers.

--- Detalhes da Amostra Aleatória #1094 ---
Barra           | Real       | Predito    | Diferença 
-------------------------------------------------------
out_v_Bus_1     | 1.0497     | 1.0491     | 0.0006
out_v_Bus_2     | 1.0434     | 1.0429     | 0.0004
out_v_Bus_3     | 1.0424     | 1.0401     | 0.0022
out_v_Bus_4     | 1.0500     | 1.0501     | 0.0001
out_v_Bus_5     | 1.0331     | 1.0305     | 0.0027
out_v_Bus_6     | 1.0193     | 1.0188     | 0.0005


In [10]:
# 1. Gera os dados LIMPOS e CORRETOS usando a função corrigida
y_t_new, y_p_new, cols_new = load_and_predict(TEST_FILE, MODEL_FILE, COLS_FILE)

# 2. Plota a tabela imediatamente com esses dados novos
plot_error_summary_table(y_t_new, y_p_new, cols_new)

Realizando predição na Rede Neural...
DEBUG: Média das Tensões Reais lidas: 1.0361 pu (Deve ser próximo de 1.0)


In [8]:
# --- 1. DADOS DE ENTRADA (Do seu data_gen.py) ---
periods = 24
hours = np.arange(24)
np.random.seed(42) # Para o gráfico ficar estático e reprodutível

# Perfis Base
load_profile_base = np.array([0.7, 0.65, 0.62, 0.60, 0.65, 0.75, 0.85, 0.95, 1.0, 1.05, 1.1, 1.08, 1.05, 1.02, 1.0, 0.98, 1.05, 1.15, 1.2, 1.18, 1.1, 1.0, 0.9, 0.8])
wind_profile_base = np.array([0.9, 0.95, 0.98, 0.92, 0.85, 0.8, 0.7, 0.6, 0.45, 0.3, 0.25, 0.35, 0.4, 0.3, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.8, 0.85, 0.88, 0.92])

n_scenarios_plot = 100 # Mais cenários para ficar denso

fig = go.Figure()

# --- 2. GERAÇÃO E PLOTAGEM DOS CENÁRIOS (Fundo) ---
for i in range(n_scenarios_plot):
    # --- Lógica do data_gen.py ---
    
    # Carga: Scale Uniforme [0.8, 1.2] + Ruído Uniforme [0.95, 1.05]
    scale_load = np.random.uniform(0.8, 1.2)
    noise_load = np.random.uniform(0.95, 1.05, periods)
    scen_load = load_profile_base * scale_load * noise_load
    
    # Eólica: Scale Uniforme [0.5, 1.2] + Ruído Uniforme [0.90, 1.10]
    scale_wind = np.random.uniform(0.5, 1.2)
    noise_wind = np.random.uniform(0.90, 1.10, periods)
    scen_wind = np.clip(wind_profile_base * scale_wind * noise_wind, 0, 1.0) # Clip físico

    # Adiciona Carga (Azul Transparente)
    fig.add_trace(go.Scatter(
        x=hours, y=scen_load, mode='lines',
        line=dict(color='rgba(31, 119, 180, 0.08)', width=1), # Bem transparente
        showlegend=False, hoverinfo='skip'
    ))

    # Adiciona Eólica (Verde Transparente)
    fig.add_trace(go.Scatter(
        x=hours, y=scen_wind, mode='lines',
        line=dict(color='rgba(44, 160, 44, 0.08)', width=1), # Bem transparente
        showlegend=False, hoverinfo='skip'
    ))

# --- 3. PLOTAGEM DAS MÉDIAS (Destaque) ---

# Média Carga
fig.add_trace(go.Scatter(
    x=hours, y=load_profile_base, mode='lines',
    name='Demanda (Perfil Típico)',
    line=dict(color='midnightblue', width=4)
))

# Média Eólica
fig.add_trace(go.Scatter(
    x=hours, y=wind_profile_base, mode='lines',
    name='Geração Eólica (Perfil Típico)',
    line=dict(color='darkgreen', width=4)
))

# --- 4. FORMATAÇÃO ---
fig.update_layout(
    title_text="Espaço de Busca Estocástico: Carga vs. Eólica",
    title_x=0.5,
    height=600, width=900,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=14),
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)', bordercolor='black', borderwidth=1),
    xaxis=dict(title="Hora do Dia", tickmode='linear', dtick=3, showgrid=True, gridcolor='lightgray'),
    yaxis=dict(title="Potência (pu)", range=[0, 1.6], showgrid=True, gridcolor='lightgray'),
)

# Adicionar anotação explicativa (Opcional)
fig.add_annotation(
    x=19, y=1.35, xref="x", yref="y",
    text="Carga Alta", showarrow=False, font=dict(color="midnightblue")
)
fig.add_annotation(
    x=19, y=0.2, xref="x", yref="y",
    text="Eólica Variável", showarrow=False, font=dict(color="darkgreen")
)

fig.show()